In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization,Activation, Input, Conv1D,GlobalAveragePooling1D , Dense
import tensorflow.keras

In [2]:
image=np.load("/home/jovyan/private/CropMappingI/preprocessing/out/Site_A/x-2017.npy")
label=np.load("/home/jovyan/private/CropMappingI/preprocessing/out/Site_A/y-2017.npy")
image=(image-np.mean(image))/np.std(image)

In [7]:
folds = [0,int(len(label)/5),2*int(len(label)/5),3*int(len(label)/5),4*int(len(label)/5),2885211]
i = np.arange(2885211)
j = np.arange(folds[0], folds[1])
k = np.delete(i,j)


images_train = image[k]
labels_train = label[k]
labels_train=to_categorical(labels_train,3)

images_val = image[j]
labels_val = label[j]
labels_val=to_categorical(labels_val,3)


In [8]:
inputlayer=Input(shape=(23,6))

conv0=Conv1D(64,kernel_size=5,strides=1, padding='same')(inputlayer)
conv0 = BatchNormalization()(conv0)
conv0 = Activation('relu')(conv0)

conv1=Conv1D(128,kernel_size=5,strides=1, padding='same')(conv0)
conv1 = BatchNormalization()(conv1)
conv1 = Activation('relu')(conv1)


conv2=Conv1D(128,kernel_size=5,strides=1, padding='same')(conv1)
conv2 = BatchNormalization()(conv2)
conv2 = Activation('relu')(conv2)

conv3=Conv1D(256,kernel_size=5,strides=1, padding='same')(conv2)
conv3 = BatchNormalization()(conv3)
conv3 = Activation('relu')(conv3)


conv4=Conv1D(512,kernel_size=5,strides=1, padding='same')(conv3)
conv4 = BatchNormalization()(conv4)
conv4 = Activation('relu')(conv4)


conv5=Conv1D(1024,kernel_size=5,strides=1, padding='same')(conv4)
conv5 = BatchNormalization()(conv5)
conv5 = Activation('relu')(conv5)




pool= GlobalAveragePooling1D()(conv5)


FC = Dense(3)(pool)
    
out = Activation('softmax',name='softm')(FC)



model=tensorflow.keras.models.Model(inputlayer,out)

model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss="categorical_crossentropy" ,metrics="accuracy")


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:

from tensorflow.keras import callbacks



check_point = callbacks.ModelCheckpoint("./save/m-{accuracy:.4f}-{epoch:02d}-{val_accuracy:.4f}.hdf5",monitor='val_accuracy', verbose=1, save_best_only=True , mode='max')
check_point2 = callbacks.ModelCheckpoint("./save/m.hdf5",monitor='accuracy', verbose=1)




model.fit(images_train,labels_train, validation_data=(images_val,labels_val),epochs=1000,batch_size=1024, callbacks=[check_point,check_point2], verbose=1,shuffle=True)


In [9]:
image_test=np.load("/home/jovyan/private/CropMappingI/preprocessing/out/Site_A/x-2018.npy")
label_test=np.load("/home/jovyan/private/CropMappingI/preprocessing/out/Site_A/y-2018.npy")
image_test=(image_test-np.mean(image_test))/np.std(image_test)

In [11]:
model.load_weights("./save/m-0.9487-06-0.9378.hdf5")

In [12]:
predictions=model.predict(image_test)

90100/90100 [==============================] - 475s 5ms/step
